In [3]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


This step is to gather data from imdb, we 

In [11]:
import pandas as pd
import json

imdb_data = pd.read_csv("data/imdb-data.csv")

result_df = imdb_data.sort_values(by='numVotes', ascending=False)

# # Top 700 movies by popularity
result_df = result_df.head(1337) #todo

result_df = result_df.rename(columns={'averageRating': 'imdb_rating'})

In [12]:
import pandas as pd
import json

filmweb_data = pd.read_csv("data/filmweb_1939_2023.csv")

def imdb_name_filmweb_mapper(name):
  try:
    filmweb_record = filmweb_data[filmweb_data['film_title_eng'] == name]
    if not filmweb_record.empty:
        return filmweb_record['rating'].iloc[0]
    return None
  except Exception as e:
    print(f"Error processing IMDb ID {name}: {str(e)}")
    return None

result_df['filmweb_rating'] = result_df['title'].apply(lambda title: imdb_name_filmweb_mapper(title))
result_df = result_df.dropna(subset=['filmweb_rating'])
result_df

,id,title,type,genres,imdb_rating,numVotes,releaseYear,filmweb_rating
83511,tt0111161,The Shawshank Redemption,movie,Drama,9.3,2991460.0,1994.0,8.8
246702,tt0468569,The Dark Knight,movie,"Action, Crime, Drama",9.0,2971515.0,2008.0,8.0
411591,tt1375666,Inception,movie,"Action, Adventure, Sci-Fi",8.8,2638507.0,2010.0,8.2
99178,tt0137523,Fight Club,movie,Drama,8.8,2418682.0,1999.0,8.3
82668,tt0109830,Forrest Gump,movie,"Drama, Romance",8.8,2340761.0,1994.0,8.5
...,...,...,...,...,...,...,...,...
80490,tt0106677,Dazed and Confused,movie,Comedy,7.6,205599.0,1993.0,6.8
152980,tt0241303,Chocolat,movie,"Drama, Romance",7.2,205364.0,2000.0,6.8
39822,tt0053604,The Apartment,movie,"Comedy, Drama, Romance",8.3,205163.0,1960.0,7.8
473250,tt15474916,Smile,movie,"Horror, Mystery, Thriller",6.5,205083.0,2022.0,6.3


In [13]:
import pandas as pd
import json

# imdb_data = pd.read_csv("data/imdb-data.csv")

# result_df = imdb_data.sort_values(by='numVotes', ascending=False)

# # Top 5000 movies by popularity
# result_df = result_df.head(50) #todo

# # To see the sorted DataFrame
# print(sorted_df_desc)

douban = pd.read_csv("data/douban-data.txt", sep='\t', engine='python')
douban = douban.dropna(subset=['imdb'])
# print(douban.describe())

def imdb_name_filmweb_mapper(imdb_id):
  if pd.isna(imdb_id):
    return None
  try:
    douban_record = douban[douban['imdb'].str.contains(imdb_id, na=False)]
    if not douban_record.empty:
        return douban_record['rating'].iloc[0]
    return None
  except Exception as e:
    print(f"Error processing IMDb ID {imdb_id}: {str(e)}")
    return None
    
result_df['douban_rating'] = result_df['id'].apply(lambda imdb_id: imdb_name_filmweb_mapper(imdb_id))

result_df = result_df.dropna(subset=['douban_rating'])
# sorted_df_desc.head(5)

def transform_douban_rating_to_imdb(rating):
    if not isinstance(rating, str) or rating.strip() == '':
        return None
    
    try:
        rating_dict = json.loads(rating.replace("'", '"'))
        if isinstance(rating_dict, dict):
            values = [float(value.strip('%')) for value in rating_dict.values()]
            average = sum(values) / len(values)
            return round(average / 10, 2)
        else:
           return float(rating)
    except Exception as e:
        # If it's not a JSON string, handle as a percentage string
        return float(rating)

# douban['rating'] = douban['rating'].apply(transform_douban_rating_to_imdb)
result_df['douban_rating'] = result_df['douban_rating'].apply(transform_douban_rating_to_imdb)
result_df = result_df[result_df['douban_rating'] > 0]
result_df

C:\Users\Fix\AppData\Local\Temp\ipykernel_32628\3807419204.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['douban_rating'] = result_df['douban_rating'].apply(transform_douban_rating_to_imdb)


,id,title,type,genres,imdb_rating,numVotes,releaseYear,filmweb_rating,douban_rating
83511,tt0111161,The Shawshank Redemption,movie,Drama,9.3,2991460.0,1994.0,8.8,9.5
246702,tt0468569,The Dark Knight,movie,"Action, Crime, Drama",9.0,2971515.0,2008.0,8.0,8.6
99178,tt0137523,Fight Club,movie,Drama,8.8,2418682.0,1999.0,8.3,9.1
82668,tt0109830,Forrest Gump,movie,"Drama, Romance",8.8,2340761.0,1994.0,8.5,9.3
83348,tt0110912,Pulp Fiction,movie,"Crime, Drama",8.9,2295044.0,1994.0,8.3,8.8
...,...,...,...,...,...,...,...,...,...
274601,tt0985694,Machete,movie,"Action, Crime, Thriller",6.6,206269.0,2010.0,6.4,9.1
256154,tt0756683,The Man from Earth,movie,"Drama, Fantasy, Mystery",7.8,206144.0,2007.0,7.2,8.4
80490,tt0106677,Dazed and Confused,movie,Comedy,7.6,205599.0,1993.0,6.8,7.6
152980,tt0241303,Chocolat,movie,"Drama, Romance",7.2,205364.0,2000.0,6.8,7.9


In [14]:
result_df.to_csv('results/ratings.csv', index=False)